# Analising Video with YOLO and MLRun


In this notebook we are going to analise the objects from a video file using the following process:

### Split and Upload Files to Minio


In [ ]:
%pip install opencv-python

In [ ]:
from video_spliter import Video_Spliter

In [ ]:
secrets = {"MINIO_URL": "", "MINIO_AK": "", "MINIO_SK": ""}

In [ ]:
v = Video_Spliter()

In [ ]:
v.split_video('mov_bbb.mp4', 'data')

### Upload Data to Minio


In [ ]:
%pip install minio

In [ ]:
import minio
client = minio.Minio(secrets["MINIO_URL"], access_key=secrets["MINIO_AK"], secret_key=secrets["MINIO_SK"])

In [ ]:
import os

def upload_local_directory_to_minio(client, local_path, bucket_name, minio_path):
  """Uploads a local directory to a Minio bucket.

  Args:
    local_path: The path to the local directory.
    bucket_name: The name of the Minio bucket.
    minio_path: The path to the directory in the Minio bucket.
  """

  assert os.path.isdir(local_path)
  for root, dirs, files in os.walk(local_path):
    for file in files:
      remote_path = os.path.join(minio_path, os.path.relpath(os.path.join(root, file), local_path))
      remote_path = remote_path.replace(os.sep, '/')

      # Upload the file.
      client.fput_object(bucket_name, remote_path, os.path.join(root, file))

upload_local_directory_to_minio(client, 'data/mov_bbb.mp4/', 'video', 'mov_bbb.mp4')


List objects in the frame order

In [ ]:
objects = client.list_objects(
    "video", recursive=True, prefix="mov_bbb.mp4/frames/"
)
file_names = [obj.object_name for obj in objects]
file_names = sorted(file_names, key=lambda string: string.lower())
file_names[-1]


### Create MLRun Model Serving

Create the custom server class: serving.py 

```python
from typing import List
import mlrun
import torch
import minio
from PIL import Image


class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        print("aa")
        torch.hub.set_dir('.')
        self.model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
        self.minio_client = minio.Minio('my-docker-ip:9000', access_key='', secret_key='', secure=False)

    def dowload_files(self, bucket: str, path_list: list):
        images = []
        for path in path_list:
            try:
                # dowload image
                self.minio_client.fget_object(bucket, path, "test.jpg")
                im = Image.open('test.jpg')
                images.append(im)
            except Exception as e:
                print(e)
                continue
        
        return images

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample."""
        images = self.dowload_files(str(body["bucket"]), body['inputs'])
        result = self.model(images)
        return str(result.tolist())
```

In [ ]:
%pip install mlrun

In [ ]:
import mlrun

project = mlrun.get_or_create_project('yolo', context="./", user_project=True)

In [ ]:
project.set_secrets(secrets=secrets, provider="kubernetes")

In [ ]:
object_tracker = mlrun.code_to_function('object_tracker', filename='image_tracking.py', kind='serving',image='smartcommunitylab/yolov5-minio:tst')

In [ ]:
# define resources
object_tracker.spec.replicas = 1
from kubernetes import client
tol = [
    client.V1Toleration(
        key='nvidia.com/gpu',
        operator='Equal',
        value='v100',
        effect='NoSchedule',
    )
]
object_tracker.with_node_selection(tolerations=tol)
object_tracker.with_limits(gpus=1,mem="70G")

In [ ]:
object_tracker.add_model('Pytorch',model_path=  ' ', class_name='TrackingModel')

In [ ]:
object_tracker.spec.graph.plot(rankdir="LR")    

Install dependencies to local testing

### Deploy Model Server on Nucleo and test

In [ ]:
import time

project.deploy_function(object_tracker)
# wait 30s after the deployment to the server statup 
time.sleep(30)

In [ ]:
import time

project.deploy_function(object_detector)
# wait 30s after the deployment to the server statup 
time.sleep(30)

In [ ]:
sample = {"inputs": file_names, "bucket": "video"}
result = object_detector.invoke(path=f"/v2/models/Pytorch/infer", body=sample)

In [ ]:
sample = {"inputs": file_names, "bucket": "video"}
result = object_tracker.invoke(path=f"/v2/models/Pytorch/infer", body=sample)

### Remake video

The OpenCV version from pip only supports the mp4v codec due to license issues. To support other codecs, you can install OpenCV from the conda-forge channel.

```conda install -c conda-forge opencv```

or via apt:

```apt-get install python3-opencv```

In [ ]:
v.frames_to_video("data/mov_bbb.mp4/", True)